# Apply RNA2seg on spatial data 

This notebook presents the workflow for applying RNA2Seg on Zarr-saved dataset. The process is structured into five main steps:

- **1-Create training patches** - Extract patches of a reasonable size to process efficiently (saved in the Zarr file). 
- **2-Initialize an RNA2segDataset**
- **3-Initialize a model RNA2Seg**
- **4-Run segmentation**: RNA2Seg is applied to the dataset to generate cell segmentations, which are then saved in a Zarr format.
- **5-Visualize segmentation**

In [1]:
import warnings
warnings.filterwarnings("ignore")

import cv2
import spatialdata as sd
from pathlib import Path

## Step 1: Create patches from Zarr files

In this step, the dataset (image + transcripts) is divided into patches of size `patch_width × patch_width` with an overlap of `patch_overlap`. This allows processing images of a manageable size while preserving spatial continuity.  

**Process** 
- The dataset, stored in Zarr format, is loaded.  
- Patches coordinates are saved as a `Shape` in the zarr: `sopa_patches_rna2seg_[patch_width]_[patch_overlap]`. 
- A `.rna2seg` directory is created to store the transcript data corresponding to each patch.  
- The transcript information for each patch is saved in CSV format for further processing.  


In [2]:
import warnings
warnings.filterwarnings("ignore")

import cv2
import spatialdata as sd
from pathlib import Path

from rna2seg.dataset_zarr.patches import create_patch_rna2seg


# MODIFY WITH YOUR PATH
merfish_zarr_path = "/media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr"

### load sdata and set path parameters 
sdata = sd.read_zarr(merfish_zarr_path)
image_key = "staining_z3"
points_key = "transcripts"
patch_width = 1200
patch_overlap = 150
min_transcripts_per_patch = 0
folder_patch_rna2seg = Path(merfish_zarr_path) / f".rna2seg_{patch_width}_{patch_overlap}"

### create patch in the sdata and precompute transcipt.csv for each patch with sopa
create_patch_rna2seg(sdata=sdata,
                    image_key=image_key,
                    points_key=points_key,
                    patch_width=patch_width,
                    patch_overlap=patch_overlap,
                    min_transcripts_per_patch=min_transcripts_per_patch,
                    folder_patch_rna2seg = folder_patch_rna2seg,
                    overwrite = True)
print(sdata)

[INFO] (sopa.patches._patches) Added 81 patche(s) to sdata['sopa_patches_rna2seg_1200_150']


[########################################] | 100% Completed | 18.11 ss
[########################################] | 100% Completed | 12.09 s
SpatialData object, with associated Zarr store: /media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr
├── Images
│     └── 'staining_z3': DataTree[cyx] (5, 9000, 9000), (5, 4500, 4500), (5, 2250, 2250), (5, 1125, 1125), (5, 562, 562)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 9) (2D points)
└── Shapes
      ├── 'Cellbound1': GeoDataFrame shape: (3258, 1) (2D shapes)
      ├── 'Cellbound1_consistent_with_nuclei': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'Cellbound1_consistent_without_nuclei': GeoDataFrame shape: (2239, 1) (2D shapes)
      ├── 'DAPI': GeoDataFrame shape: (2377, 1) (2D shapes)
      ├── 'DAPI_consistent_in_cell': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'DAPI_consistent_not_in_cell': GeoDataFrame shape: (1370, 1) (2D shapes)
      ├── 'sopa_patches_rna

## Step 2: Initialize a RNA2segDataset

In [3]:
import albumentations as A
from rna2seg.dataset_zarr import RNA2segDataset

transform_resize  = A.Compose([
    A.Resize(width=512, height=512, interpolation=cv2.INTER_NEAREST),
])

dataset = RNA2segDataset(
    sdata=sdata,
    channels_dapi= ["DAPI"],
    channels_cellbound=["Cellbound1"],
    patch_width = patch_width,
    patch_overlap = patch_overlap,
    gene_column="gene",
    transform_resize = transform_resize,
    patch_dir=folder_patch_rna2seg
)

default shape_patch_key set to sopa_patches_rna2seg_1200_150
100%|██████████| 81/81 [00:01<00:00, 80.64it/s]
Number of valid patches: 64


## Step 3: Initialize a model RNA2seg

By default, the trained RNA2seg model is downloaded from Hugging Face. You can also specify `pretrained_model` to use your own RNA2seg model.

In [4]:
from rna2seg.models import RNA2seg


device = "cpu"


rna2seg = RNA2seg(
    device,
    net='unet',
    flow_threshold = 0.9,
    cellbound_flow_threshold = 0.4,
    pretrained_model = "default_pretrained"
)

No module named 'vmunet'
VMUnet not loaded
initiaisation of CPnet
Initiaisation of ChannelInvariantNet


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights from /home/tom/.cache/huggingface/hub/models--aliceblondel--RNA2seg/snapshots/26f9afe181ea58f808ecd9c0732c327936020b18/rna2seg.pt


## Step 4: Run segmentation 

We will run the segmentation on the entire dataset and save the results as a shape in the Zarr file.

In [5]:
from tqdm import tqdm
from rna2seg.utils import save_shapes2zarr

for i in tqdm(range(len(dataset))):
    input_dict = dataset[i]
    rna2seg.run(
        path_temp_save=folder_patch_rna2seg,
        input_dict=input_dict
    )

  2%|▏         | 1/64 [00:04<05:02,  4.80s/it]

47 cells detected


  3%|▎         | 2/64 [00:07<03:29,  3.38s/it]

45 cells detected


  5%|▍         | 3/64 [00:08<02:38,  2.60s/it]

...

97 cells detected


 98%|█████████▊| 63/64 [02:47<00:03,  3.07s/it]

109 cells detected


100%|██████████| 64/64 [02:50<00:00,  2.67s/it]

95 cells detected


### Add segmented cell in the spatialdata as shape
The segmentation of each patch are stored in ```folder_patch_rna2seg``` and can be add in the spataildata as follow : 

In [6]:
segmentation_shape_name = "test_rnas2eg"
save_shapes2zarr(dataset, segmentation_shape_name)

100%|██████████| 64/64 [00:00<00:00, 347.80it/s]


len(list_all_cells) 3930


Resolving conflicts: 100%|██████████| 5854/5854 [00:01<00:00, 4499.42it/s]


Added 3011 cell boundaries in sdata['test_rnas2eg']


## Step 5: Visualize segmentation

Now, we visualize the results.
The following code extracts and plots three images: RNA transcripts, a selected staining image, and the corresponding segmentation.
Ensure that `staining_id` is valid to avoid indexing errors, and check that `bounds` are within dataset limits.

In [ ]:
dataset.st_segmentation.sdata

In [ ]:
import matplotlib.pyplot as plt

# Parameters
key_segmentation="test_rnas2eg"
bounds = [2000, 3000, 3000, 4000,] # xmin, ymin, xmax, ymax
staining_id = 1 # Staining to plot

# Get images
rna = dataset.get_rna_img(bounds, key_transcripts=points_key,)
image = dataset.get_staining_img(bounds)
seg = dataset.get_segmentation_img(bounds, key_cell=key_segmentation)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(rna)
axes[0].set_title("RNA")
axes[0].axis("off")

axes[1].imshow(image[staining_id], cmap="grey")
axes[1].set_title("Staining")
axes[1].axis("off")

axes[2].imshow(seg)
axes[2].set_title("Segmentation")
axes[2].axis("off")

plt.show()